# Temat testów
Czy istnieje istotna statystycznie różnica w poprawności/pewności odpowiedzi w zależności od cech osobowościowych z ankiety?

In [6]:
import pandas as pd

from functions.tests import conduct_chi2test_for_many, conduct_posthoc_test
from functions.set_preparations import prepare_row
from functions.others import choose_dominant_kind

In [7]:
df_corr = pd.read_csv(
    '/Users/martasolarz/Studies/Thesis/data/correctness.csv',
    usecols=['ID', '1a_corr', '1b_corr', '2a_corr', '2b_corr', '3a_corr', '3b_corr', 'Wynik', '1a_trud', '1b_trud',
             '2a_trud', '2b_trud', '3a_trud', '3b_trud', 'Trud_all']
).fillna('NO_VAL')

df_corr = df_corr[df_corr.ID != 6]  # odrzucony uczestnik ze względu na metryki eye trackingowe

a = df_corr[['1a_corr', '2a_corr', '3a_corr']]
b = df_corr[['1b_corr', '2b_corr', '3b_corr']]
df_corr['SumCorr_a'] = a.apply(lambda x: x.sum(), axis=1)
df_corr['SumCorr_b'] = b.apply(lambda x: x.sum(), axis=1)
gr1 = df_corr[['1a_corr', '1b_corr']]
gr2 = df_corr[['2a_corr', '2b_corr']]
gr3 = df_corr[['3a_corr', '3b_corr']]
df_corr['SumCorr_1'] = gr1.apply(lambda x: x.sum(), axis=1)
df_corr['SumCorr_2'] = gr2.apply(lambda x: x.sum(), axis=1)
df_corr['SumCorr_3'] = gr3.apply(lambda x: x.sum(), axis=1)

a = df_corr[['1a_trud', '2a_trud', '3a_trud']]
b = df_corr[['1b_trud', '2b_trud', '3b_trud']]
df_corr['SumTrud_a'] = a.apply(lambda x: x.sum(), axis=1)
df_corr['SumTrud_b'] = b.apply(lambda x: x.sum(), axis=1)
gr1 = df_corr[['1a_trud', '1b_trud']]
gr2 = df_corr[['2a_trud', '2b_trud']]
gr3 = df_corr[['3a_trud', '3b_trud']]
df_corr['SumTrud_1'] = gr1.apply(lambda x: x.sum(), axis=1)
df_corr['SumTrud_2'] = gr2.apply(lambda x: x.sum(), axis=1)
df_corr['SumTrud_3'] = gr3.apply(lambda x: x.sum(), axis=1)

df_quest = pd.read_csv(
    '/Users/martasolarz/Studies/Thesis/data/survey.csv',
).fillna('NO_VAL')

df_quest = df_quest[df_quest.ID != 6]  # odrzucony uczestnik ze względu na metryki eye trackingowe

In [8]:
# zagregowane kolumny - podział na tych co pisali i nie pisali danej matury:
df_quest['MAT'] = df_quest['MAT_roz'].apply(lambda x: x if x == 'NO_VAL' else 'VAL')
df_quest['POL'] = df_quest['JP_roz'].apply(lambda x: x if x == 'NO_VAL' else 'VAL')
df_quest['HIST'] = df_quest['HIST_roz'].apply(lambda x: x if x == 'NO_VAL' else 'VAL')
df_quest['Typ_uczenia'] = df_quest.apply(choose_dominant_kind, axis=1)
df_quest.head()

,ID,Płeć,Rok_urodzenia,Kierunek_studiów,Rok_studiów,Specjalność,Wady_wzroku,Daltonizm,Ilość_snu,JP_podst,...,Rank_map,Rank_txt,Rank_plt,Rank_tab,Data,Godzina,MAT,POL,HIST,Typ_uczenia
0,1,M,2002,"Geografia, w ramach studiów międzyobszarowych",2,SPOŁ-EKO,0,1,6,90 - 100%,...,3,4,1,2,2023-05-24,11:44:20,VAL,VAL,NO_VAL,Kinestetyk
1,2,M,2003,Gospodarka przestrzenna,1,NO_VAL,1,1,<6,50 - 69%,...,2,3,1,4,2023-05-24,12:21:03,VAL,VAL,NO_VAL,Brak dominującego
2,3,M,2001,Geografia,3,GEOINF,0,1,8,50 - 69%,...,1,4,3,2,2023-05-24,12:41:09,VAL,NO_VAL,NO_VAL,Wzrokowiec
3,4,K,2000,Geografia,3,GEOINF,0,1,8,50 - 69%,...,2,4,3,1,2023-05-24,13:15:20,NO_VAL,VAL,NO_VAL,Brak dominującego
4,5,K,2002,Geografia,3,GEOINF,0,1,7,50 - 69%,...,2,4,1,3,2023-05-24,13:35:17,NO_VAL,NO_VAL,NO_VAL,Wzrokowiec


In [9]:
df_merge = df_corr.merge(df_quest, on='ID').set_index('ID')
df_merge.tail()

,1a_trud,1b_trud,2a_trud,2b_trud,3a_trud,3b_trud,Trud_all,Wynik,1a_corr,1b_corr,...,Rank_map,Rank_txt,Rank_plt,Rank_tab,Data,Godzina,MAT,POL,HIST,Typ_uczenia
ID,,,,,,,,,,,,,,,,,,,,,
42,2,1,2,1,2,2,10,5,1,1,...,2,4,1,3,2023-06-07,11:42:55,NO_VAL,NO_VAL,NO_VAL,Wzrokowiec
43,1,2,2,2,3,3,13,5,1,1,...,2,3,1,2,2023-06-07,13:04:51,NO_VAL,NO_VAL,NO_VAL,Brak dominującego
44,1,1,3,1,2,3,11,3,1,1,...,2,1,2,1,2023-06-07,14:13:38,VAL,NO_VAL,NO_VAL,Brak dominującego
45,2,1,2,2,2,2,11,3,1,1,...,2,4,3,1,2023-06-07,14:27:37,VAL,NO_VAL,NO_VAL,Słuchowiec
46,1,3,2,2,2,3,13,4,0,1,...,1,3,3,3,2023-06-07,15:19:52,NO_VAL,NO_VAL,NO_VAL,Brak dominującego


Przygotowanie data setów dla wszystkich testowanych przypadków:

1. Poprawność vs dane osobowe
2. Pewność vs dane osobowe

In [11]:
# zsumowany wynik wszystkie grafiki
df_wynik = df_merge.copy()
df_wynik.rename(columns={'Wynik': 'corr', 'Trud_all': 'trud'}, inplace=True)

# zsumowany wynik - grafiki A
df_sumA = df_merge.copy()
df_sumA.rename(columns={'SumCorr_a': 'corr', 'SumTrud_a': 'trud'}, inplace=True)

# zsumowany wynik - grafiki B
df_sumB = df_merge.copy()
df_sumB.rename(columns={'SumCorr_b': 'corr', 'SumTrud_b': 'trud'}, inplace=True)

# zsumowany wynik - grafiki 1
df_sum1 = df_merge.copy()
df_sum1.rename(columns={'SumCorr_1': 'corr', 'SumTrud_1': 'trud'}, inplace=True)

# zsumowany wynik - grafiki 2
df_sum2 = df_merge.copy()
df_sum2.rename(columns={'SumCorr_2': 'corr', 'SumTrud_2': 'trud'}, inplace=True)

# zsumowany wynik - grafiki 3
df_sum3 = df_merge.copy()
df_sum3.rename(columns={'SumCorr_3': 'corr', 'SumTrud_3': 'trud'}, inplace=True)
df_sum3

,1a_trud,1b_trud,2a_trud,2b_trud,3a_trud,3b_trud,Trud_all,Wynik,1a_corr,1b_corr,...,Rank_map,Rank_txt,Rank_plt,Rank_tab,Data,Godzina,MAT,POL,HIST,Typ_uczenia
ID,,,,,,,,,,,,,,,,,,,,,
1,3,1,4,3,4,4,19,4,1,1,...,3,4,1,2,2023-05-24,11:44:20,VAL,VAL,NO_VAL,Kinestetyk
2,1,2,3,1,2,4,13,5,1,1,...,2,3,1,4,2023-05-24,12:21:03,VAL,VAL,NO_VAL,Brak dominującego
3,2,1,3,1,1,1,9,4,1,1,...,1,4,3,2,2023-05-24,12:41:09,VAL,NO_VAL,NO_VAL,Wzrokowiec
4,3,2,2,1,3,5,16,5,1,1,...,2,4,3,1,2023-05-24,13:15:20,NO_VAL,VAL,NO_VAL,Brak dominującego
5,4,2,3,2,3,3,17,6,1,1,...,2,4,1,3,2023-05-24,13:35:17,NO_VAL,NO_VAL,NO_VAL,Wzrokowiec
7,2,2,2,1,4,3,14,6,1,1,...,2,4,1,3,2023-05-25,13:46:15,VAL,NO_VAL,VAL,Brak dominującego
11,2,1,3,2,1,2,11,2,1,0,...,1,4,1,3,2023-05-25,14:12:48,NO_VAL,NO_VAL,NO_VAL,Wzrokowiec
12,1,1,3,3,2,2,12,2,1,1,...,1,4,2,3,2023-05-25,14:47:58,VAL,NO_VAL,NO_VAL,Brak dominującego
13,2,4,2,1,1,3,13,5,1,1,...,1,4,2,3,2023-05-25,15:13:03,VAL,NO_VAL,NO_VAL,Wzrokowiec


In [12]:
# wszystkie grafiki - zagregowane do jednej kolumny
df_answers = df_corr.copy()

df_list = [df_quest] * 6
df_all = pd.concat(df_list, ignore_index=True)

popr = []
for col in df_answers.columns.tolist():
    if col.endswith('corr'):
        popr.append(df_answers[col])

trud = []
for col in df_answers.columns.tolist():
    if col.endswith('trud'):
        trud.append(df_answers[col])

df_all["trud"] = prepare_row(trud)
df_all["corr"] = prepare_row(popr)

In [13]:
# grafiki A - zagregowane do jednej kolumny
df_list = [df_quest] * 3
df_a = pd.concat(df_list, ignore_index=True)

popr = []
for col in df_answers.columns.tolist():
    if col.endswith('a_corr'):
        popr.append(df_answers[col])

trud = []
for col in df_answers.columns.tolist():
    if col.endswith('a_trud'):
        trud.append(df_answers[col])

df_a["trud"] = prepare_row(trud)
df_a["corr"] = prepare_row(popr)

In [14]:
# grafiki B - zagregowane do jednej kolumny
df_list = [df_quest] * 3
df_b = pd.concat(df_list, ignore_index=True)

popr = []
for col in df_answers.columns.tolist():
    if col.endswith('b_corr'):
        popr.append(df_answers[col])

trud = []
for col in df_answers.columns.tolist():
    if col.endswith('b_trud'):
        trud.append(df_answers[col])

df_b["trud"] = prepare_row(trud)
df_b["corr"] = prepare_row(popr)

In [27]:
# grafiki 1,2,3 - zagregowane do jednej kolumny
df_list = [df_quest] * 2
df_1 = pd.concat(df_list, ignore_index=True)

popr1 = []
for col in df_answers.columns.tolist():
    if col.startswith('1') and col.endswith('_corr'):
        popr1.append(df_answers[col])

trud1 = []
for col in df_answers.columns.tolist():
    if col.startswith('1') and col.endswith('_trud'):
        trud1.append(df_answers[col])
df_1["trud"] = prepare_row(trud1)
df_1["corr"] = prepare_row(popr1)

####

df_list = [df_quest] * 2
df_2 = pd.concat(df_list, ignore_index=True)

popr2 = []
for col in df_answers.columns.tolist():
    if col.startswith('2') and col.endswith('_corr'):
        popr2.append(df_answers[col])

trud2 = []
for col in df_answers.columns.tolist():
    if col.startswith('2') and col.endswith('_trud'):
        trud2.append(df_answers[col])

df_2["trud"] = prepare_row(trud2)
df_2["corr"] = prepare_row(popr2)

#####

df_list = [df_quest] * 2
df_3 = pd.concat(df_list, ignore_index=True)

popr3 = []
for col in df_answers.columns.tolist():
    if col.startswith('3') and col.endswith('_corr'):
        popr3.append(df_answers[col])

trud3 = []
for col in df_answers.columns.tolist():
    if col.startswith('3') and col.endswith('_trud'):
        trud3.append(df_answers[col])

df_3["trud"] = prepare_row(trud3)
df_3["corr"] = prepare_row(popr3)

df_1

,ID,Płeć,Rok_urodzenia,Kierunek_studiów,Rok_studiów,Specjalność,Wady_wzroku,Daltonizm,Ilość_snu,JP_podst,...,Rank_plt,Rank_tab,Data,Godzina,MAT,POL,HIST,Typ_uczenia,trud,corr
0,1,M,2002,"Geografia, w ramach studiów międzyobszarowych",2,SPOŁ-EKO,0,1,6,90 - 100%,...,1,2,2023-05-24,11:44:20,VAL,VAL,NO_VAL,Kinestetyk,3,1
1,2,M,2003,Gospodarka przestrzenna,1,NO_VAL,1,1,<6,50 - 69%,...,1,4,2023-05-24,12:21:03,VAL,VAL,NO_VAL,Brak dominującego,1,1
2,3,M,2001,Geografia,3,GEOINF,0,1,8,50 - 69%,...,3,2,2023-05-24,12:41:09,VAL,NO_VAL,NO_VAL,Wzrokowiec,2,1
3,4,K,2000,Geografia,3,GEOINF,0,1,8,50 - 69%,...,3,1,2023-05-24,13:15:20,NO_VAL,VAL,NO_VAL,Brak dominującego,3,1
4,5,K,2002,Geografia,3,GEOINF,0,1,7,50 - 69%,...,1,3,2023-05-24,13:35:17,NO_VAL,NO_VAL,NO_VAL,Wzrokowiec,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,42,K,2003,Geografia,1,FIZ,0,1,7,70 - 89%,...,1,3,2023-06-07,11:42:55,NO_VAL,NO_VAL,NO_VAL,Wzrokowiec,1,1
76,43,M,2003,Gospodarka przestrzenna,1,NO_VAL,1,1,8,50 - 69%,...,1,2,2023-06-07,13:04:51,NO_VAL,NO_VAL,NO_VAL,Brak dominującego,2,1
77,44,K,2003,Geografia,1,FIZ,1,1,<6,50 - 69%,...,2,1,2023-06-07,14:13:38,VAL,NO_VAL,NO_VAL,Brak dominującego,1,1
78,45,M,2003,Geografia,1,GEOINF,1,1,8<,30 - 49%,...,3,1,2023-06-07,14:27:37,VAL,NO_VAL,NO_VAL,Słuchowiec,1,1


**H0:** Nie ma istotnej statystycznie zależności między poprawnością/trudnością odpowiedzi a wartością cechy X.
**H1:** Istnieje istotna statystycznie zależność między poprawnością/trudnością odpowiedzi a wartością cechy X.
**Poziom istotności:** $\alpha=0.05$

In [28]:
ALPHA = 0.05

In [29]:
df_quest.columns

Index(['ID', 'Płeć', 'Rok_urodzenia', 'Kierunek_studiów', 'Rok_studiów',
       'Specjalność', 'Wady_wzroku', 'Daltonizm', 'Ilość_snu', 'JP_podst',
       'MAT_podst', 'JP_roz', 'MAT_roz', 'FIZ_roz', 'HIST_roz', 'Samopocz',
       'Stres', 'Zmęcz', 'Czyt_map', 'Podziel_uwg', 'Analit_myśl', 'Spostrzeg',
       'Czyt_ze_zroz', 'Wzrokowiec', 'Słuchowiec', 'Kinestetyk',
       'Quest1_after', 'Quest2_after', 'Rank_map', 'Rank_txt', 'Rank_plt',
       'Rank_tab', 'Data', 'Godzina', 'MAT', 'POL', 'HIST', 'Typ_uczenia'],
      dtype='object')

In [30]:
dfs = [df_wynik, df_sumA, df_sumB, df_sum1, df_sum2, df_sum3, df_all, df_a, df_b, df_1, df_2, df_3]
names = {
    1: 'Zsumowane wszystkie grafiki:',
    2: 'Zsumowane grafiki A:',
    3: 'Zsumowane grafiki B:',
    4: 'Zsumowane grafiki 1:',
    5: 'Zsumowane grafiki 2:',
    6: 'Zsumowane grafiki 3:',
    7: 'Wszystkie grafiki:',
    8: 'Grafiki A:',
    9: 'Grafiki B:',
    10: 'Grafiki 1:',
    11: 'Grafiki 2:',
    12: 'Grafiki 3:',
}

### TESTY:

Poprawność

In [31]:
# płeć
conduct_chi2test_for_many(dfs, ALPHA, names, 'corr', 'Płeć')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Płeć  K  M
corr      
0     1  0
2     2  1
3     6  2
4     5  3
5     4  9
6     5  2
Wartość statystyki chi-kwadrat: 6.283503367902344
P-value: 0.27960197071433124
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Płeć  K  M
corr      
0     1  0
1     6  2
2     9  7
3     7  8
Wartość statystyki chi-kwadrat: 2.47229326513214
P-value: 0.48031912676829214
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki B:
Płeć   K   M
corr        
0      1   0
1      4   2
2     11  10
3      7   5
Wartość statystyki chi-kwadrat: 1.1740348313238333
P-value: 0.7592383281795241
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki 1:
Płeć   K   M
corr        
0      2   0
1      3   1
2     18  16
Wartość statystyki chi-kwadrat: 2.2686926432977286
P-value: 0.32163229670274396
Brak podst

In [32]:
# rok urodzenia
conduct_chi2test_for_many(dfs, ALPHA, names, 'corr', 'Rok_urodzenia')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Rok_urodzenia  2000  2001  2002  2003  2004
corr                                       
0                 0     1     0     0     0
2                 0     0     1     2     0
3                 0     3     1     4     0
4                 0     1     5     2     0
5                 1     0     3     8     1
6                 0     3     1     1     2
Wartość statystyki chi-kwadrat: 26.40690192160781
P-value: 0.1527898007412733
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Rok_urodzenia  2000  2001  2002  2003  2004
corr                                       
0                 0     1     0     0     0
1                 0     1     2     5     0
2                 0     3     5     8     0
3                 1     3     4     4     3
Wartość statystyki chi-kwadrat: 12.757464349376116
P-value: 0.38691296398079306
Brak podstaw do odrzucenia H0.
------------------

In [33]:
conduct_posthoc_test(df_1, 'corr', 'Rok_urodzenia')

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
  2000   2001  -0.3125 0.6038 -0.9189 0.2939  False
  2000   2002  -0.0455 0.9995 -0.6426 0.5517  False
  2000   2003  -0.0588 0.9986 -0.6471 0.5294  False
  2000   2004      0.0    1.0 -0.6601 0.6601  False
  2001   2002    0.267 0.0482  0.0014 0.5327   True
  2001   2003   0.2537 0.0389  0.0086 0.4988   True
  2001   2004   0.3125 0.1707 -0.0745 0.6995  False
  2002   2003  -0.0134 0.9998 -0.2346 0.2078  False
  2002   2004   0.0455  0.997 -0.3269 0.4178  False
  2003   2004   0.0588 0.9907 -0.2992 0.4168  False
---------------------------------------------------


In [34]:
# kierunek studiów
conduct_chi2test_for_many(dfs, ALPHA, names, 'corr', 'Kierunek_studiów')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Kierunek_studiów  Geografia  Geografia, w ramach studiów międzyobszarowych  \
corr                                                                         
0                         1                                              0   
2                         2                                              0   
3                         7                                              0   
4                         7                                              1   
5                         9                                              1   
6                         6                                              1   

Kierunek_studiów  Gospodarka przestrzenna  
corr                                       
0                                       0  
2                                       1  
3                                       1  
4                                       0  
5                                 

In [35]:
conduct_posthoc_test(df_sum3, 'corr', 'Kierunek_studiów')
conduct_posthoc_test(df_3, 'corr', 'Kierunek_studiów')

                                       Multiple Comparison of Means - Tukey HSD, FWER=0.05                                       
                    group1                                        group2                    meandiff p-adj   lower  upper  reject
---------------------------------------------------------------------------------------------------------------------------------
                                    Geografia Geografia, w ramach studiów międzyobszarowych   1.0625 0.0607 -0.0395 2.1645  False
                                    Geografia                       Gospodarka przestrzenna  -0.1375 0.9227 -1.0151 0.7401  False
Geografia, w ramach studiów międzyobszarowych                       Gospodarka przestrzenna     -1.2 0.0847 -2.5328 0.1328  False
---------------------------------------------------------------------------------------------------------------------------------
                                       Multiple Comparison of Means - Tukey HSD, FWER=0.05

In [36]:
# rok studiów
conduct_chi2test_for_many(dfs, ALPHA, names, 'corr', 'Rok_studiów')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Rok_studiów  1  2  3
corr                
0            1  0  0
2            3  0  0
3            4  1  3
4            3  4  1
5            9  3  1
6            3  0  4
Wartość statystyki chi-kwadrat: 15.606154111588893
P-value: 0.11147544155399011
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Rok_studiów  1  2  3
corr                
0            1  0  0
1            6  1  1
2            9  4  3
3            7  3  5
Wartość statystyki chi-kwadrat: 3.138687600644123
P-value: 0.7912496415904829
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki B:
Rok_studiów   1  2  3
corr                 
0             1  0  0
1             4  0  2
2            10  8  3
3             8  0  4
Wartość statystyki chi-kwadrat: 9.965493443754312
P-value: 0.1261127070604207
Brak podstaw do odrzucenia H0.
------------------------------

In [37]:
# specjalność
conduct_chi2test_for_many(dfs, ALPHA, names, 'corr', 'Specjalność')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Specjalność  FIZ   GEOINF   NO_VAL  SPOŁ-EKO
corr                                        
0               1        0       0         0
2               1        1       1         0
3               1        5       1         1
4               1        3       0         4
5               4        4       3         2
6               1        5       0         1
Wartość statystyki chi-kwadrat: 16.081654456654455
P-value: 0.37664570282661447
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Specjalność  FIZ   GEOINF   NO_VAL  SPOŁ-EKO
corr                                        
0               1        0       0         0
1               2        4       1         1
2               3        5       3         5
3               3        9       1         2
Wartość statystyki chi-kwadrat: 7.554166666666667
P-value: 0.579622037631938
Brak podstaw do odrzucenia H0.
-----

In [38]:
# wady wzroku
conduct_chi2test_for_many(dfs, ALPHA, names, 'corr', 'Wady_wzroku')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Wady_wzroku  0  1
corr             
0            0  1
2            2  1
3            2  6
4            4  4
5            4  9
6            4  3
Wartość statystyki chi-kwadrat: 3.957570207570208
P-value: 0.5555408744707073
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Wady_wzroku  0   1
corr              
0            0   1
1            3   5
2            6  10
3            7   8
Wartość statystyki chi-kwadrat: 1.0069444444444446
P-value: 0.7995716113771797
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki B:
Wady_wzroku  0   1
corr              
0            0   1
1            3   3
2            8  13
3            5   7
Wartość statystyki chi-kwadrat: 0.9623015873015874
P-value: 0.8103727681036947
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki 1:
Wady_wzroku   0

In [39]:
# ilość snu
conduct_chi2test_for_many(dfs, ALPHA, names, 'corr', 'Ilość_snu')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Ilość_snu  6  7  8  8<  <6
corr                      
0          1  0  0   0   0
2          1  0  0   2   0
3          1  0  0   2   5
4          3  4  1   0   0
5          3  3  2   2   3
6          2  2  1   0   2
Wartość statystyki chi-kwadrat: 26.07878232878233
P-value: 0.1632256743168377
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Ilość_snu  6  7  8  8<  <6
corr                      
0          1  0  0   0   0
1          1  0  0   3   4
2          6  5  1   1   3
3          3  4  3   2   3
Wartość statystyki chi-kwadrat: 15.109175084175085
P-value: 0.23552021082362976
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki B:
Ilość_snu  6  7  8  8<  <6
corr                      
0          1  0  0   0   0
1          2  0  0   3   1
2          4  6  3   3   5
3          4  3  1   0   4
Wartość statystyki chi-kw

In [40]:
conduct_posthoc_test(df_2, 'corr', 'Ilość_snu')

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     6      7   0.2879 0.2374 -0.0981 0.6738  False
     6      8   0.4545 0.0939 -0.0468 0.9559  False
     6     8<  -0.0455 0.9984 -0.4812 0.3903  False
     6     <6   0.2545 0.3282 -0.1206 0.6297  False
     7      8   0.1667 0.8949 -0.3493 0.6827  False
     7     8<  -0.3333 0.2489 -0.7859 0.1192  False
     7     <6  -0.0333 0.9993 -0.4279 0.3612  False
     8     8<     -0.5 0.0967 -1.0543 0.0543  False
     8     <6     -0.2 0.8057  -0.708  0.308  False
    8<     <6      0.3  0.331 -0.1434 0.7434  False
---------------------------------------------------


In [41]:
# matura - j.pol - podst
conduct_chi2test_for_many(dfs, ALPHA, names, 'corr', 'JP_podst')

-----------------------------------------------
Zsumowane wszystkie grafiki:
JP_podst  30 - 49%  50 - 69%  70 - 89%  90 - 100%
corr                                             
0                1         0         0          0
2                1         2         0          0
3                1         2         4          1
4                1         2         3          2
5                0         5         6          2
6                1         2         4          0
Wartość statystyki chi-kwadrat: 15.372880513152007
P-value: 0.4249063194554209
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
JP_podst  30 - 49%  50 - 69%  70 - 89%  90 - 100%
corr                                             
0                1         0         0          0
1                1         4         2          1
2                2         4         8          2
3                1         5         7          2
Wartość statystyki chi-kwadrat: 9.3001508295

In [44]:
# matura - mat - podst
conduct_chi2test_for_many(dfs, ALPHA, names, 'corr', 'MAT_podst')

-----------------------------------------------
Zsumowane wszystkie grafiki:
MAT_podst  30 - 49%  50 - 69%  70 - 89%  90 - 100%
corr                                              
0                 0         0         1          0
2                 0         0         2          1
3                 0         1         1          6
4                 1         1         4          2
5                 0         0         4          9
6                 0         1         0          6
Wartość statystyki chi-kwadrat: 17.046703296703296
P-value: 0.31608015346220947
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
MAT_podst  30 - 49%  50 - 69%  70 - 89%  90 - 100%
corr                                              
0                 0         0         1          0
1                 0         1         3          4
2                 1         1         4         10
3                 0         1         4         10
Wartość statystyki chi-kwadra

In [45]:
conduct_posthoc_test(df_3, 'corr', 'MAT_podst')

  Multiple Comparison of Means - Tukey HSD, FWER=0.05   
 group1    group2  meandiff p-adj   lower  upper  reject
--------------------------------------------------------
30 - 49%  50 - 69%   0.3333 0.8348  -0.708 1.3747  False
30 - 49%  70 - 89%   0.3333 0.7874 -0.6053  1.272  False
30 - 49% 90 - 100%    0.625 0.2891 -0.2954 1.5454  False
50 - 69%  70 - 89%      0.0    1.0 -0.5821 0.5821  False
50 - 69% 90 - 100%   0.2917 0.5112 -0.2606 0.8439  False
70 - 89% 90 - 100%   0.2917 0.0851 -0.0272 0.6105  False
--------------------------------------------------------


In [46]:
# matura - j.pol - roz
conduct_chi2test_for_many(dfs, ALPHA, names, 'corr', 'JP_roz')

-----------------------------------------------
Zsumowane wszystkie grafiki:
JP_roz  20 - 49%  50 - 70%  NO_VAL  powyżej 70%
corr                                           
0              0         0       1            0
2              0         0       3            0
3              0         1       6            1
4              0         1       5            2
5              1         3       9            0
6              0         0       7            0
Wartość statystyki chi-kwadrat: 11.0206782464847
P-value: 0.7511271503844899
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
JP_roz  20 - 49%  50 - 70%  NO_VAL  powyżej 70%
corr                                           
0              0         0       1            0
1              0         1       6            1
2              1         2      11            2
3              0         2      13            0
Wartość statystyki chi-kwadrat: 4.025806451612903
P-value: 0.9097058053721

In [47]:
# matura - mat - roz
conduct_chi2test_for_many(dfs, ALPHA, names, 'corr', 'MAT_roz')

-----------------------------------------------
Zsumowane wszystkie grafiki:
MAT_roz  20 - 49%  50 - 70%  NO_VAL  poniżej 20%  powyżej 70%
corr                                                         
0               0         0       1            0            0
2               0         1       1            1            0
3               0         2       3            2            1
4               0         2       4            1            1
5               2         3       6            0            2
6               0         2       3            1            1
Wartość statystyki chi-kwadrat: 9.874440374440375
P-value: 0.9703919389516331
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
MAT_roz  20 - 49%  50 - 70%  NO_VAL  poniżej 20%  powyżej 70%
corr                                                         
0               0         0       1            0            0
1               0         2       4            2            0
2

In [48]:
# matura - hist - roz
conduct_chi2test_for_many(dfs, ALPHA, names, 'corr', 'HIST_roz')

-----------------------------------------------
Zsumowane wszystkie grafiki:
HIST_roz  20 - 49%  NO_VAL  powyżej 70%
corr                                   
0                0       1            0
2                0       3            0
3                0       8            0
4                1       7            0
5                0      11            2
6                0       6            1
Wartość statystyki chi-kwadrat: 7.202380952380952
P-value: 0.7062107889594078
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
HIST_roz  20 - 49%  NO_VAL  powyżej 70%
corr                                   
0                0       1            0
1                0       8            0
2                1      15            0
3                0      12            3
Wartość statystyki chi-kwadrat: 6.791666666666666
P-value: 0.34054431725484147
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki B:
HIST_r

In [49]:
# samopoczucie
conduct_chi2test_for_many(dfs, ALPHA, names, 'corr', 'Samopocz')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Samopocz  2  3  4  5
corr                
0         1  0  0  0
2         1  1  0  1
3         1  3  2  2
4         0  0  3  5
5         2  2  8  1
6         0  0  7  0
Wartość statystyki chi-kwadrat: 30.932336182336183
P-value: 0.008969827796645308
Istnieją podstawy do odrzucenia H0, przyjmujemy hipotezę H1.
-----------------------------------------------
Zsumowane grafiki A:
Samopocz  2  3   4  5
corr                 
0         1  0   0  0
1         2  4   0  2
2         2  1   7  6
3         0  1  13  1
Wartość statystyki chi-kwadrat: 29.371296296296297
P-value: 0.0005608843981853312
Istnieją podstawy do odrzucenia H0, przyjmujemy hipotezę H1.
-----------------------------------------------
Zsumowane grafiki B:
Samopocz  2  3  4  5
corr                
0         1  0  0  0
1         1  1  2  2
2         1  4  9  7
3         2  1  9  0
Wartość statystyki chi-kwadrat: 15.00793650793651
P-value: 0.0907182048738

In [51]:
conduct_posthoc_test(df_wynik, 'corr', 'Samopocz')
conduct_posthoc_test(df_sumA, 'corr', 'Samopocz')
conduct_posthoc_test(df_sum3, 'corr', 'Samopocz')
conduct_posthoc_test(df_all, 'corr', 'Samopocz')
conduct_posthoc_test(df_a, 'corr', 'Samopocz')
conduct_posthoc_test(df_3, 'corr', 'Samopocz')

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     2      3      0.5 0.8948 -1.4123  2.4123  False
     2      4      2.0 0.0084   0.421   3.579   True
     2      5   0.6667 0.7393 -1.0948  2.4282  False
     3      4      1.5  0.044    0.03    2.97   True
     3      5   0.1667  0.993 -1.4978  1.8311  False
     4      5  -1.3333 0.0361 -2.6009 -0.0657   True
----------------------------------------------------
Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     2      3      0.3  0.853 -0.7075  1.3075  False
     2      4     1.45 0.0002  0.6181  2.2819   True
     2      5   0.6889 0.2072 -0.2391  1.6169  False
     3      4     1.15 0.0016  0.3755  1.9245   True
     3      5   0.3889 0.6343  -0.488  1.2658  False
     4      5  -0.7611 0.0203 -1.4289 -0.0933 

In [52]:
# zmęczenie
conduct_chi2test_for_many(dfs, ALPHA, names, 'corr', 'Zmęcz')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Zmęcz  1  2  3  4  5
corr                
0      0  0  0  1  0
2      2  0  0  1  0
3      0  2  3  1  2
4      1  4  2  1  0
5      3  3  4  2  1
6      0  3  1  3  0
Wartość statystyki chi-kwadrat: 23.070004070004074
P-value: 0.28537027733583586
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Zmęcz  1  2  3  4  5
corr                
0      0  0  0  1  0
1      2  1  1  2  2
2      2  6  7  1  0
3      2  5  2  5  1
Wartość statystyki chi-kwadrat: 15.974074074074075
P-value: 0.19242640519976809
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki B:
Zmęcz  1  2  3  4  5
corr                
0      0  0  0  1  0
1      2  1  2  1  0
2      3  7  4  4  3
3      1  4  4  3  0
Wartość statystyki chi-kwadrat: 9.301587301587304
P-value: 0.6769778081553945
Brak podstaw do odrzucenia H0.
----------------------------------

In [53]:
# czytanie map
conduct_chi2test_for_many(dfs, ALPHA, names, 'corr', 'Czyt_map')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Czyt_map  3  4  5
corr             
0         0  1  0
2         1  0  2
3         1  7  0
4         1  5  2
5         1  7  5
6         1  5  1
Wartość statystyki chi-kwadrat: 10.167032967032966
P-value: 0.4259632464411981
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Czyt_map  3   4  5
corr              
0         0   1  0
1         2   4  2
2         1  11  4
3         2   9  4
Wartość statystyki chi-kwadrat: 2.4400000000000004
P-value: 0.8751212607961629
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki B:
Czyt_map  3   4  5
corr              
0         0   1  0
1         1   3  2
2         3  13  5
3         1   8  3
Wartość statystyki chi-kwadrat: 1.2666666666666668
P-value: 0.9734638923917822
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki 1:
Czyt_map  3   

In [54]:
# podzielność uwagi
conduct_chi2test_for_many(dfs, ALPHA, names, 'corr', 'Podziel_uwg')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Podziel_uwg  1  2  3  4  5
corr                      
0            0  0  1  0  0
2            0  1  1  1  0
3            0  1  3  2  2
4            1  2  3  1  1
5            1  2  3  4  3
6            0  1  3  1  2
Wartość statystyki chi-kwadrat: 7.786753590325017
P-value: 0.9931871605303147
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Podziel_uwg  1  2  3  4  5
corr                      
0            0  0  1  0  0
1            0  2  4  1  1
2            1  2  5  6  2
3            1  3  4  2  5
Wartość statystyki chi-kwadrat: 8.330026455026456
P-value: 0.7588374568200759
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki B:
Podziel_uwg  1  2  3  4  5
corr                      
0            0  0  1  0  0
1            0  1  1  3  1
2            2  5  7  2  5
3            0  1  5  4  2
Wartość statystyki chi-kwad

In [55]:
# analityczne myślenie
conduct_chi2test_for_many(dfs, ALPHA, names, 'corr', 'Analit_myśl')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Analit_myśl  2  3  4  5
corr                   
0            0  0  1  0
2            0  2  0  1
3            1  0  6  1
4            0  4  4  0
5            0  2  6  5
6            0  3  2  2
Wartość statystyki chi-kwadrat: 18.8615867563236
P-value: 0.22007476752021932
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Analit_myśl  2  3  4  5
corr                   
0            0  0  1  0
1            1  2  4  1
2            0  5  9  2
3            0  4  5  6
Wartość statystyki chi-kwadrat: 9.19457735247209
P-value: 0.4195104670780957
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki B:
Analit_myśl  2  3   4  5
corr                    
0            0  0   1  0
1            0  2   2  2
2            1  5  11  4
3            0  4   5  3
Wartość statystyki chi-kwadrat: 3.2140958456747932
P-value: 0.9551982534540059
Bra

In [56]:
# spostrzegawczość
conduct_chi2test_for_many(dfs, ALPHA, names, 'corr', 'Spostrzeg')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Spostrzeg  2  3  4  5
corr                 
0          0  0  0  1
2          0  1  0  2
3          1  4  2  1
4          0  2  5  1
5          2  1  7  3
6          1  3  3  0
Wartość statystyki chi-kwadrat: 17.81517502105737
P-value: 0.2725070261956324
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Spostrzeg  2  3  4  5
corr                 
0          0  0  0  1
1          1  3  2  2
2          0  5  7  4
3          3  3  8  1
Wartość statystyki chi-kwadrat: 10.45944741532977
P-value: 0.31458547055404834
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki B:
Spostrzeg  2  3   4  5
corr                  
0          0  0   0  1
1          0  3   0  3
2          3  4  12  2
3          1  4   5  2
Wartość statystyki chi-kwadrat: 14.348102877514641
P-value: 0.11047017673363393
Brak podstaw do odrzucenia H0.
---------

In [57]:
conduct_posthoc_test(df_all, 'corr', 'Spostrzeg')

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     2      3  -0.1098 0.7369  -0.388  0.1683  False
     2      4  -0.0172 0.9983 -0.2819  0.2476  False
     2      5    -0.25 0.1215 -0.5417  0.0417  False
     3      4   0.0927 0.5632 -0.0917   0.277  False
     3      5  -0.1402 0.3593 -0.3615  0.0812  False
     4      5  -0.2328 0.0183 -0.4371 -0.0286   True
----------------------------------------------------


In [58]:
# czytanie ze zrozumieniem
conduct_chi2test_for_many(dfs, ALPHA, names, 'corr', 'Czyt_ze_zroz')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Czyt_ze_zroz  1  2  3  4  5
corr                       
0             0  0  0  1  0
2             0  1  1  0  1
3             1  2  2  2  1
4             0  2  0  4  2
5             1  1  2  6  3
6             0  1  1  4  1
Wartość statystyki chi-kwadrat: 10.569388011614903
P-value: 0.956634737506914
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Czyt_ze_zroz  1  2  3  4  5
corr                       
0             0  0  0  1  0
1             1  2  3  0  2
2             0  3  1  8  4
3             1  2  2  8  2
Wartość statystyki chi-kwadrat: 11.760270774976659
P-value: 0.46511970837383954
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki B:
Czyt_ze_zroz  1  2  3  4  5
corr                       
0             0  0  0  1  0
1             0  2  1  2  1
2             2  4  3  8  4
3             0  1  2  6  3
Warto

In [59]:
# wzrokowiec
conduct_chi2test_for_many(dfs, ALPHA, names, 'corr', 'Wzrokowiec')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Wzrokowiec  2  3  4  5
corr                  
0           0  0  1  0
2           0  0  1  2
3           0  2  3  3
4           1  2  3  2
5           0  3  4  6
6           1  2  3  1
Wartość statystyki chi-kwadrat: 8.457003314146172
P-value: 0.9041579133602895
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Wzrokowiec  2  3  4  5
corr                  
0           0  0  1  0
1           0  1  3  4
2           1  4  5  6
3           1  4  6  4
Wartość statystyki chi-kwadrat: 3.7478835978835985
P-value: 0.9272090299966425
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki B:
Wzrokowiec  2  3  4  5
corr                  
0           0  0  1  0
1           0  1  2  3
2           1  5  8  7
3           1  3  4  4
Wartość statystyki chi-kwadrat: 2.873015873015873
P-value: 0.9691171812074049
Brak podstaw do odrzucenia H

In [60]:
# słuchowiec
conduct_chi2test_for_many(dfs, ALPHA, names, 'corr', 'Słuchowiec')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Słuchowiec  1  2  3  4  5
corr                     
0           0  0  0  0  1
2           1  1  1  0  0
3           1  2  3  2  0
4           1  1  2  1  3
5           0  3  6  2  2
6           1  1  3  2  0
Wartość statystyki chi-kwadrat: 17.077729809872668
P-value: 0.647920711018114
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Słuchowiec  1  2  3  4  5
corr                     
0           0  0  0  0  1
1           1  3  3  1  0
2           2  2  6  3  3
3           1  3  6  3  2
Wartość statystyki chi-kwadrat: 9.24325396825397
P-value: 0.6820248528873389
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki B:
Słuchowiec  1  2  3  4  5
corr                     
0           0  0  0  0  1
1           2  1  2  1  0
2           1  5  7  3  5
3           1  2  6  3  0
Wartość statystyki chi-kwadrat: 14.7188208616780

In [61]:
conduct_posthoc_test(df_sum2, 'corr', 'Słuchowiec')
conduct_posthoc_test(df_2, 'corr', 'Słuchowiec')

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     1      2     1.25 0.0815 -0.1023 2.6023  False
     1      3      1.1 0.1036 -0.1427 2.3427  False
     1      4   0.9286 0.3216 -0.4555 2.3127  False
     1      5   0.6667 0.6658 -0.7588 2.0921  False
     2      3    -0.15 0.9914 -1.1168 0.8168  False
     2      4  -0.3214 0.9262 -1.4643 0.8214  False
     2      5  -0.5833 0.6278 -1.7759 0.6093  False
     3      4  -0.1714  0.988 -1.1822 0.8394  False
     3      5  -0.4333 0.7692    -1.5 0.6334  False
     4      5  -0.2619 0.9721 -1.4905 0.9667  False
---------------------------------------------------
Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     1      2    0.625 0.0104  0.1054 1.1446   True
     1      3     0.55 0.0158  0.0725 1.0275   True
     1      

In [62]:
# kinestetyk
conduct_chi2test_for_many(dfs, ALPHA, names, 'corr', 'Kinestetyk')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Kinestetyk  1  2  3  4  5
corr                     
0           0  1  0  0  0
2           1  0  0  2  0
3           3  3  0  2  0
4           1  2  1  3  1
5           2  7  2  2  0
6           2  4  0  1  0
Wartość statystyki chi-kwadrat: 15.77240297828533
P-value: 0.7306398067351932
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Kinestetyk  1  2  3  4  5
corr                     
0           0  1  0  0  0
1           2  2  0  4  0
2           3  6  3  3  1
3           4  8  0  3  0
Wartość statystyki chi-kwadrat: 10.975708061002178
P-value: 0.5310016553083297
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki B:
Kinestetyk  1  2  3  4  5
corr                     
0           0  1  0  0  0
1           3  1  0  2  0
2           4  8  1  7  1
3           2  7  2  1  0
Wartość statystyki chi-kwadrat: 10.37566137566

In [63]:
# MAT - pisał / nie pisał matury rozszerzonej
conduct_chi2test_for_many(dfs, ALPHA, names, 'corr', 'MAT')

-----------------------------------------------
Zsumowane wszystkie grafiki:
MAT   NO_VAL  VAL
corr             
0          1    0
2          1    2
3          3    5
4          4    4
5          6    7
6          3    4
Wartość statystyki chi-kwadrat: 1.6698116698116698
P-value: 0.8926808101752923
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
MAT   NO_VAL  VAL
corr             
0          1    0
1          4    4
2          6   10
3          7    8
Wartość statystyki chi-kwadrat: 1.6835016835016834
P-value: 0.6406079569226228
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki B:
MAT   NO_VAL  VAL
corr             
0          1    0
1          1    5
2         11   10
3          5    7
Wartość statystyki chi-kwadrat: 3.6844636844636853
P-value: 0.29761397694125696
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki 1:
MAT   NO_VAL  VAL
corr   

In [64]:
# POL - pisał / nie pisał matury rozszerzonej
conduct_chi2test_for_many(dfs, ALPHA, names, 'corr', 'POL')

-----------------------------------------------
Zsumowane wszystkie grafiki:
POL   NO_VAL  VAL
corr             
0          1    0
2          3    0
3          6    2
4          5    3
5          9    4
6          7    0
Wartość statystyki chi-kwadrat: 4.764267990074441
P-value: 0.44532128745838995
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
POL   NO_VAL  VAL
corr             
0          1    0
1          6    2
2         11    5
3         13    2
Wartość statystyki chi-kwadrat: 1.7443249701314216
P-value: 0.6271247147520325
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki B:
POL   NO_VAL  VAL
corr             
0          1    0
1          6    0
2         14    7
3         10    2
Wartość statystyki chi-kwadrat: 3.6798088410991636
P-value: 0.2981793034790027
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki 1:
POL   NO_VAL  VAL
corr    

In [65]:
# HIST - pisał / nie pisał matury rozszerzonej
conduct_chi2test_for_many(dfs, ALPHA, names, 'corr', 'HIST')

-----------------------------------------------
Zsumowane wszystkie grafiki:
HIST  NO_VAL  VAL
corr             
0          1    0
2          3    0
3          8    0
4          7    1
5         11    2
6          6    1
Wartość statystyki chi-kwadrat: 1.9505494505494503
P-value: 0.8559442969077931
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
HIST  NO_VAL  VAL
corr             
0          1    0
1          8    0
2         15    1
3         12    3
Wartość statystyki chi-kwadrat: 2.916666666666667
P-value: 0.40465279495160555
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki B:
HIST  NO_VAL  VAL
corr             
0          1    0
1          6    0
2         18    3
3         11    1
Wartość statystyki chi-kwadrat: 1.243386243386243
P-value: 0.7426183968451716
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki 1:
HIST  NO_VAL  VAL
corr     

In [66]:
# dominujący typ uczenia poznawczego
conduct_chi2test_for_many(dfs, ALPHA, names, 'corr', 'Typ_uczenia')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Typ_uczenia  Brak dominującego  Kinestetyk  Słuchowiec  Wzrokowiec
corr                                                              
0                            0           0           1           0
2                            1           0           0           2
3                            1           0           1           6
4                            2           2           2           2
5                            3           0           2           8
6                            3           0           0           4
Wartość statystyki chi-kwadrat: 19.252081252081254
P-value: 0.20253723959819042
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Typ_uczenia  Brak dominującego  Kinestetyk  Słuchowiec  Wzrokowiec
corr                                                              
0                            0           0           1           0
1     

In [67]:
conduct_posthoc_test(df_2, 'corr', 'Typ_uczenia')

        Multiple Comparison of Means - Tukey HSD, FWER=0.05        
      group1        group2   meandiff p-adj   lower   upper  reject
-------------------------------------------------------------------
Brak dominującego Kinestetyk    -0.35 0.4713 -0.9831  0.2831  False
Brak dominującego Słuchowiec  -0.4333  0.042 -0.8554 -0.0113   True
Brak dominującego Wzrokowiec     -0.1 0.8339 -0.4117  0.2117  False
       Kinestetyk Słuchowiec  -0.0833 0.9877 -0.7507   0.584  False
       Kinestetyk Wzrokowiec     0.25  0.698 -0.3537  0.8537  False
       Słuchowiec Wzrokowiec   0.3333 0.1011 -0.0431  0.7098  False
-------------------------------------------------------------------


Podsumowanie - poprawność:
Istotne statystycznie:
- kierunek studiów: zsumowane grafiki 3 
- rok urodzenia: dla grafik 1 (2001 vs 2003 oraz 2001 vs 2002 --> na korzyść 2003, 2002, generalnie młodsi lepiej)
- kierunek studiów: zsumowane grafiki 3, grafik 3 (geografia vs geografia w ramach MISMaP --> na korzyść MISMaP)
- ilość snu: dla grafik 2 (bardzo słabo się różnicuje w postHoc)
- matura podstawowa mat: grafiki 3 (bardzo słabo się różnicuje w postHoc), ale generalnie lepsze wyniki matury skutkowały większym prawdopodobieństwem poprawnej odpowiedzi
- samopoczucie: zsumowane grafiki, zsumowane grafiki A, zsumowane grafiki 3, grafiki wszystkie, grafiki A, grafiki 3 --> test postHoc wykazuje że różnice istotne statystycznie nie są dla skrajnych wartości sampoczucia, ale dla sąsiednich np. 3-4 
- spostrzegawczość: wszystkie grafiki (4 vs 5)
- słuchowiec: zsumowane grafiki 2, grafiki 2 (1 vs 2, 1 vs 3)
- dominujący typ uczenia poznawczego: grafiki 2 (brak dominującego vs słuchowiec)


Trudność/pewność:

In [73]:
# płeć
conduct_chi2test_for_many(dfs, ALPHA, names, 'trud', 'Płeć')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Płeć  K  M
trud      
9     0  1
10    2  2
11    1  2
12    1  1
13    2  3
14    5  1
15    1  2
16    3  2
17    3  0
18    1  1
19    1  1
20    2  1
22    1  0
Wartość statystyki chi-kwadrat: 8.35464620630861
P-value: 0.7568383089489292
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Płeć  K  M
trud      
5     2  0
6     5  8
7     3  3
8     8  4
9     2  0
10    2  0
11    0  2
12    1  0
Wartość statystyki chi-kwadrat: 10.358712046691586
P-value: 0.16914041919463849
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki B:
Płeć  K  M
trud      
3     0  1
4     2  2
5     1  4
6     3  0
7     4  4
8     6  4
9     1  1
10    5  0
11    1  0
12    0  1
Wartość statystyki chi-kwadrat: 12.583120204603578
P-value: 0.1823944389132879
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Z

In [74]:
# rok urodzenia
conduct_chi2test_for_many(dfs, ALPHA, names, 'trud', 'Rok_urodzenia')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Rok_urodzenia  2000  2001  2002  2003  2004
trud                                       
9                 0     1     0     0     0
10                0     1     0     3     0
11                0     0     0     3     0
12                0     0     0     2     0
13                0     0     2     2     1
14                0     2     3     1     0
15                0     0     1     1     1
16                1     1     1     1     1
17                0     1     1     1     0
18                0     2     0     0     0
19                0     0     1     1     0
20                0     0     2     1     0
22                0     0     0     1     0
Wartość statystyki chi-kwadrat: 43.89587046939988
P-value: 0.6416391200564315
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Rok_urodzenia  2000  2001  2002  2003  2004
trud                                     

In [75]:
# kierunek studiów
conduct_chi2test_for_many(dfs, ALPHA, names, 'trud', 'Kierunek_studiów')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Kierunek_studiów  Geografia  Geografia, w ramach studiów międzyobszarowych  \
trud                                                                         
9                         1                                              0   
10                        4                                              0   
11                        3                                              0   
12                        2                                              0   
13                        3                                              0   
14                        4                                              1   
15                        2                                              0   
16                        4                                              0   
17                        3                                              0   
18                        2                                      

In [76]:
# rok studiów
conduct_chi2test_for_many(dfs, ALPHA, names, 'trud', 'Rok_studiów')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Rok_studiów  1  2  3
trud                
9            0  0  1
10           3  0  1
11           3  0  0
12           2  0  0
13           4  1  0
14           2  2  2
15           2  1  0
16           2  1  2
17           2  0  1
18           0  0  2
19           1  1  0
20           1  2  0
22           1  0  0
Wartość statystyki chi-kwadrat: 27.16183574879227
P-value: 0.29696618338935205
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Rok_studiów   1  2  3
trud                 
5             2  0  0
6            10  1  2
7             3  2  1
8             5  3  4
9             2  0  0
10            0  1  1
11            0  1  1
12            1  0  0
Wartość statystyki chi-kwadrat: 13.252043849869937
P-value: 0.5067805369863833
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki B:
Rok_studiów  1  2  3
trud     

In [77]:
# specjalność
conduct_chi2test_for_many(dfs, ALPHA, names, 'trud', 'Specjalność')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Specjalność  FIZ   GEOINF   NO_VAL  SPOŁ-EKO
trud                                        
9               0        1       0         0
10              1        1       0         2
11              1        2       0         0
12              1        1       0         0
13              2        1       2         0
14              0        4       1         1
15              1        1       1         0
16              1        2       1         1
17              1        2       0         0
18              0        2       0         0
19              1        0       0         1
20              0        1       0         2
22              0        0       0         1
Wartość statystyki chi-kwadrat: 31.333333333333332
P-value: 0.6901106579212577
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Specjalność  FIZ   GEOINF   NO_VAL  SPOŁ-EKO
trud                    

In [78]:
# wady wzroku
conduct_chi2test_for_many(dfs, ALPHA, names, 'trud', 'Wady_wzroku')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Wady_wzroku  0  1
trud             
9            1  0
10           3  1
11           1  2
12           1  1
13           1  4
14           2  4
15           0  3
16           3  2
17           1  2
18           1  1
19           1  1
20           1  2
22           0  1
Wartość statystyki chi-kwadrat: 8.402777777777779
P-value: 0.7529160311661997
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Wady_wzroku  0  1
trud             
5            1  1
6            6  7
7            1  5
8            5  7
9            0  2
10           2  0
11           1  1
12           0  1
Wartość statystyki chi-kwadrat: 6.746794871794872
P-value: 0.4557111008276795
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki B:
Wady_wzroku  0  1
trud             
3            1  0
4            3  1
5            1  4
6            2  1
7        

In [79]:
conduct_posthoc_test(df_sum2, 'trud', 'Wady_wzroku')

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj  lower  upper  reject
--------------------------------------------------
     0      1   0.4792 0.3783 -0.609 1.5673  False
--------------------------------------------------


In [80]:
# ilość snu
conduct_chi2test_for_many(dfs, ALPHA, names, 'trud', 'Ilość_snu')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Ilość_snu  6  7  8  8<  <6
trud                      
9          0  0  1   0   0
10         2  1  0   0   1
11         0  0  0   2   1
12         0  0  0   2   0
13         0  1  1   1   2
14         2  3  0   0   1
15         1  1  0   0   1
16         1  0  1   0   3
17         1  1  0   1   0
18         1  0  1   0   0
19         2  0  0   0   0
20         1  2  0   0   0
22         0  0  0   0   1
Wartość statystyki chi-kwadrat: 60.58181818181818
P-value: 0.10506497942114681
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Ilość_snu  6  7  8  8<  <6
trud                      
5          0  1  0   1   0
6          3  1  2   3   4
7          0  3  0   2   1
8          5  1  2   0   4
9          1  1  0   0   0
10         0  2  0   0   0
11         2  0  0   0   0
12         0  0  0   0   1
Wartość statystyki chi-kwadrat: 33.622895622895626
P-value: 0.2135653

In [81]:
conduct_posthoc_test(df_sum3, 'trud', 'Ilość_snu')
conduct_posthoc_test(df_all, 'trud', 'Ilość_snu')
conduct_posthoc_test(df_3, 'trud', 'Ilość_snu')

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     6      7   0.3636 0.9942 -2.2266 2.9539  False
     6      8   0.1136    1.0 -3.2512 3.4785  False
     6     8<  -1.4697 0.6039 -4.3945 1.4551  False
     6     <6   0.8636 0.8597 -1.6544 3.3817  False
     7      8    -0.25 0.9996 -3.7131 3.2131  False
     7     8<  -1.8333 0.4264 -4.8707  1.204  False
     7     <6      0.5 0.9821 -2.1479 3.1479  False
     8     8<  -1.5833 0.7378 -5.3033 2.1367  False
     8     <6     0.75 0.9687 -2.6594 4.1594  False
    8<     <6   2.3333 0.1842 -0.6427 5.3093  False
---------------------------------------------------
Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     6      7   -0.069 0.9968 -0.6145 0.4765  False
     6      8  -0.2727 0.8277 -0.9813 0.4359  False
     6     8

In [82]:
# matura - j.pol - podst
conduct_chi2test_for_many(dfs, ALPHA, names, 'trud', 'JP_podst')

-----------------------------------------------
Zsumowane wszystkie grafiki:
JP_podst  30 - 49%  50 - 69%  70 - 89%  90 - 100%
trud                                             
9                0         1         0          0
10               1         0         3          0
11               1         2         0          0
12               1         0         1          0
13               0         2         3          0
14               0         1         4          1
15               0         0         2          1
16               1         2         2          0
17               1         2         0          0
18               0         1         1          0
19               0         1         0          1
20               0         1         0          2
22               0         0         1          0
Wartość statystyki chi-kwadrat: 37.94992458521871
P-value: 0.38052539920429856
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafi

In [83]:
# matura - mat - podst
conduct_chi2test_for_many(dfs, ALPHA, names, 'trud', 'MAT_podst')

-----------------------------------------------
Zsumowane wszystkie grafiki:
MAT_podst  30 - 49%  50 - 69%  70 - 89%  90 - 100%
trud                                              
9                 0         0         1          0
10                0         0         1          3
11                0         1         1          1
12                0         0         2          0
13                0         0         1          4
14                0         1         2          3
15                0         0         0          3
16                0         1         2          2
17                0         0         1          2
18                0         0         0          2
19                0         0         0          2
20                1         0         1          1
22                0         0         0          1
Wartość statystyki chi-kwadrat: 31.97222222222222
P-value: 0.6606400249174549
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Z

In [84]:
conduct_posthoc_test(df_sumB, 'trud', 'MAT_podst')
conduct_posthoc_test(df_sum1, 'trud', 'MAT_podst')

   Multiple Comparison of Means - Tukey HSD, FWER=0.05   
 group1    group2  meandiff p-adj   lower   upper  reject
---------------------------------------------------------
30 - 49%  50 - 69%     -4.0 0.3634 -10.5211 2.5211  False
30 - 49%  70 - 89%  -4.3333 0.2123 -10.2114 1.5447  False
30 - 49% 90 - 100%   -3.625 0.3417  -9.3889 2.1389  False
50 - 69%  70 - 89%  -0.3333 0.9947  -3.9787 3.3121  False
50 - 69% 90 - 100%    0.375 0.9912  -3.0833 3.8333  False
70 - 89% 90 - 100%   0.7083 0.7752  -1.2883  2.705  False
---------------------------------------------------------
  Multiple Comparison of Means - Tukey HSD, FWER=0.05   
 group1    group2  meandiff p-adj   lower  upper  reject
--------------------------------------------------------
30 - 49%  50 - 69%     -3.0 0.1899 -6.9459 0.9459  False
30 - 49%  70 - 89%     -3.0 0.1238 -6.5568 0.5568  False
30 - 49% 90 - 100%  -2.5417  0.221 -6.0294 0.9461  False
50 - 69%  70 - 89%      0.0    1.0 -2.2058 2.2058  False
50 - 69% 90 - 100%   

In [85]:
# matura - j.pol - roz
conduct_chi2test_for_many(dfs, ALPHA, names, 'trud', 'JP_roz')

-----------------------------------------------
Zsumowane wszystkie grafiki:
JP_roz  20 - 49%  50 - 70%  NO_VAL  powyżej 70%
trud                                           
9              0         0       1            0
10             1         0       3            0
11             0         0       3            0
12             0         0       2            0
13             0         1       4            0
14             0         1       3            2
15             0         0       3            0
16             0         1       4            0
17             0         0       3            0
18             0         0       2            0
19             0         0       1            1
20             0         1       2            0
22             0         1       0            0
Wartość statystyki chi-kwadrat: 34.282437275985664
P-value: 0.5504283747008301
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
JP_roz  20 - 49%  50 - 7

In [86]:
# matura - mat - roz
conduct_chi2test_for_many(dfs, ALPHA, names, 'trud', 'MAT_roz')

-----------------------------------------------
Zsumowane wszystkie grafiki:
MAT_roz  20 - 49%  50 - 70%  NO_VAL  poniżej 20%  powyżej 70%
trud                                                         
9               0         1       0            0            0
10              0         0       2            1            1
11              0         0       1            2            0
12              0         0       1            1            0
13              1         1       2            0            1
14              1         1       3            0            1
15              0         2       1            0            0
16              0         1       3            1            0
17              0         0       3            0            0
18              0         2       0            0            0
19              0         2       0            0            0
20              0         0       1            0            2
22              0         0       1            0       

In [87]:
# matura - hist - roz
conduct_chi2test_for_many(dfs, ALPHA, names, 'trud', 'HIST_roz')

-----------------------------------------------
Zsumowane wszystkie grafiki:
HIST_roz  20 - 49%  NO_VAL  powyżej 70%
trud                                   
9                0       1            0
10               0       4            0
11               0       3            0
12               0       2            0
13               0       5            0
14               0       4            2
15               0       3            0
16               0       4            1
17               0       3            0
18               0       2            0
19               0       2            0
20               1       2            0
22               0       1            0
Wartość statystyki chi-kwadrat: 21.77777777777778
P-value: 0.5925284985006025
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
HIST_roz  20 - 49%  NO_VAL  powyżej 70%
trud                                   
5                0       2            0
6                0      1

In [88]:
# samopoczucie
conduct_chi2test_for_many(dfs, ALPHA, names, 'trud', 'Samopocz')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Samopocz  2  3  4  5
trud                
9         0  0  0  1
10        0  0  3  1
11        0  0  0  3
12        1  0  1  0
13        0  2  2  1
14        1  1  2  2
15        1  0  2  0
16        0  0  5  0
17        1  1  1  0
18        0  0  2  0
19        1  0  1  0
20        0  1  1  1
22        0  1  0  0
Wartość statystyki chi-kwadrat: 43.87777777777777
P-value: 0.17219319174624023
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Samopocz  2  3  4  5
trud                
5         0  0  1  1
6         2  1  4  6
7         1  1  3  1
8         1  3  8  0
9         1  0  0  1
10        0  0  2  0
11        0  0  2  0
12        0  1  0  0
Wartość statystyki chi-kwadrat: 24.373219373219374
P-value: 0.2753285792501282
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki B:
Samopocz  2  3  4  5
trud               

In [89]:
conduct_posthoc_test(df_sumB, 'trud', 'Samopocz')
conduct_posthoc_test(df_2, 'trud', 'Samopocz')
conduct_posthoc_test(df_3, 'trud', 'Samopocz')

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     2      3   0.1333 0.9996 -3.2559 3.5226  False
     2      4     -1.1 0.7164 -3.8986 1.6986  False
     2      5  -1.9778 0.3354 -5.0998 1.1442  False
     3      4  -1.2333 0.5844 -3.8387  1.372  False
     3      5  -2.1111 0.2349 -5.0611 0.8389  False
     4      5  -0.8778 0.7202 -3.1244 1.3689  False
---------------------------------------------------
Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     2      3     0.35 0.8246 -0.7172  1.4172  False
     2      4   -0.975 0.0242 -1.8562 -0.0938   True
     2      5  -0.7333 0.2125 -1.7164  0.2497  False
     3      4   -1.325 0.0004 -2.1454 -0.5046   True
     3      5  -1.0833 0.0157 -2.0122 -0.1545   True
     4      5   0.2417 0.8062 -0.4657  0.9491  False
---

In [90]:
# zmęczenie
conduct_chi2test_for_many(dfs, ALPHA, names, 'trud', 'Zmęcz')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Zmęcz  1  2  3  4  5
trud                
9      0  1  0  0  0
10     1  0  3  0  0
11     1  2  0  0  0
12     1  1  0  0  0
13     1  3  0  0  1
14     2  1  2  1  0
15     0  1  2  0  0
16     0  0  1  3  1
17     0  1  0  1  1
18     0  1  0  1  0
19     0  0  2  0  0
20     0  1  0  2  0
22     0  0  0  1  0
Wartość statystyki chi-kwadrat: 52.39259259259259
P-value: 0.30750533015684944
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Zmęcz  1  2  3  4  5
trud                
5      1  1  0  0  0
6      3  6  4  0  0
7      2  1  2  0  1
8      0  2  2  6  2
9      0  0  1  1  0
10     0  2  0  0  0
11     0  0  1  1  0
12     0  0  0  1  0
Wartość statystyki chi-kwadrat: 33.43589743589743
P-value: 0.2200645044699645
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki B:
Zmęcz  1  2  3  4  5
trud                

In [91]:
conduct_posthoc_test(df_all, 'trud', 'Zmęcz')

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     1      2     0.25 0.7721 -0.3407 0.8407  False
     1      3   0.3111  0.627 -0.2989 0.9212  False
     1      4   0.8889 0.0011  0.2663 1.5115   True
     1      5      0.5   0.47 -0.3353 1.3353  False
     2      3   0.0611 0.9974 -0.4447 0.5669  False
     2      4   0.6389 0.0077   0.118 1.1598   True
     2      5     0.25 0.8962 -0.5126 1.0126  False
     3      4   0.5778 0.0306   0.035 1.1206   True
     3      5   0.1889 0.9631 -0.5888 0.9665  False
     4      5  -0.3889 0.6555 -1.1765 0.3987  False
---------------------------------------------------


In [92]:
# czytanie map
conduct_chi2test_for_many(dfs, ALPHA, names, 'trud', 'Czyt_map')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Czyt_map  3  4  5
trud             
9         0  0  1
10        0  3  1
11        1  1  1
12        2  0  0
13        0  5  0
14        0  4  2
15        0  1  2
16        1  3  1
17        0  3  0
18        0  1  1
19        0  2  0
20        1  1  1
22        0  1  0
Wartość statystyki chi-kwadrat: 32.146666666666675
P-value: 0.12339707788575943
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Czyt_map  3  4  5
trud             
5         0  2  0
6         2  6  5
7         1  4  1
8         1  8  3
9         0  2  0
10        1  1  0
11        0  1  1
12        0  1  0
Wartość statystyki chi-kwadrat: 8.65128205128205
P-value: 0.8527200082450626
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki B:
Czyt_map  3  4  5
trud             
3         0  0  1
4         0  3  1
5         2  2  1
6         1  1  1
7       

In [93]:
# podzielność uwagi
conduct_chi2test_for_many(dfs, ALPHA, names, 'trud', 'Podziel_uwg')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Podziel_uwg  1  2  3  4  5
trud                      
9            0  0  1  0  0
10           0  0  1  2  1
11           0  0  2  1  0
12           0  2  0  0  0
13           0  3  0  1  1
14           0  1  0  3  2
15           0  0  1  0  2
16           1  0  1  2  1
17           0  0  3  0  0
18           0  1  1  0  0
19           0  0  1  0  1
20           1  0  2  0  0
22           0  0  1  0  0
Wartość statystyki chi-kwadrat: 55.97751322751323
P-value: 0.200388470048393
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Podziel_uwg  1  2  3  4  5
trud                      
5            0  2  0  0  0
6            0  1  5  5  2
7            0  2  1  0  3
8            1  1  4  4  2
9            0  0  2  0  0
10           1  0  1  0  0
11           0  1  0  0  1
12           0  0  1  0  0
Wartość statystyki chi-kwadrat: 38.13593813593813
P-value: 0.0958544763

In [94]:
conduct_posthoc_test(df_sum3, 'trud', 'Podziel_uwg')
conduct_posthoc_test(df_3, 'trud', 'Podziel_uwg')

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     1      2  -3.8571 0.1226 -8.3588 0.6445  False
     1      3  -3.2857 0.1942 -7.5299 0.9585  False
     1      4  -3.8889 0.1031  -8.278 0.5002  False
     1      5   -2.875 0.3559 -7.3137 1.5637  False
     2      3   0.5714 0.9688 -2.0276 3.1705  False
     2      4  -0.0317    1.0 -2.8612 2.7977  False
     2      5   0.9821  0.866 -1.9237  3.888  False
     3      4  -0.6032 0.9498  -3.002 1.7956  False
     3      5   0.4107 0.9892 -2.0777 2.8991  False
     4      5   1.0139 0.8212 -1.7143 3.7421  False
---------------------------------------------------
Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     1      2  -1.9286 0.0211 -3.6572 -0.1999   True
     1      3  -1.6429 0.0472 -3.2727 -0.0131   True
     1 

In [95]:
# analityczne myślenie
conduct_chi2test_for_many(dfs, ALPHA, names, 'trud', 'Analit_myśl')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Analit_myśl  2  3  4  5
trud                   
9            0  0  1  0
10           0  0  4  0
11           1  1  1  0
12           0  1  1  0
13           0  3  1  1
14           0  2  3  1
15           0  1  0  2
16           0  0  1  4
17           0  0  3  0
18           0  1  1  0
19           0  0  2  0
20           0  2  0  1
22           0  0  1  0
Wartość statystyki chi-kwadrat: 46.436292752082224
P-value: 0.11414343557949386
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Analit_myśl  2  3  4  5
trud                   
5            0  2  0  0
6            1  3  7  2
7            0  1  4  1
8            0  3  3  6
9            0  1  1  0
10           0  1  1  0
11           0  0  2  0
12           0  0  1  0
Wartość statystyki chi-kwadrat: 18.82768303820935
P-value: 0.5961931152253142
Brak podstaw do odrzucenia H0.
----------------------------------

In [96]:
# spostrzegawczość
conduct_chi2test_for_many(dfs, ALPHA, names, 'trud', 'Spostrzeg')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Spostrzeg  2  3  4  5
trud                 
9          0  0  1  0
10         0  1  1  2
11         0  0  1  2
12         1  1  0  0
13         0  2  2  1
14         0  1  4  1
15         0  1  1  1
16         1  2  2  0
17         0  1  1  1
18         1  0  1  0
19         0  1  1  0
20         0  1  2  0
22         1  0  0  0
Wartość statystyki chi-kwadrat: 34.26292335115864
P-value: 0.551367309230494
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Spostrzeg  2  3  4  5
trud                 
5          0  1  1  0
6          0  2  6  5
7          1  1  3  1
8          1  5  4  2
9          0  2  0  0
10         0  0  2  0
11         1  0  1  0
12         1  0  0  0
Wartość statystyki chi-kwadrat: 28.248320307143835
P-value: 0.13324816377042786
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki B:
Spostrzeg  2  3 

In [97]:
conduct_posthoc_test(df_sum2, 'trud', 'Spostrzeg')
conduct_posthoc_test(df_all, 'trud', 'Spostrzeg')
conduct_posthoc_test(df_a, 'trud', 'Spostrzeg')
conduct_posthoc_test(df_3, 'trud', 'Spostrzeg')

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     2      3   0.4545 0.9681 -2.2327 3.1418  False
     2      4   0.7059 0.8789 -1.8518 3.2635  False
     2      5    0.875 0.8369 -1.9434 3.6934  False
     3      4   0.2513  0.981 -1.5296 2.0323  False
     3      5   0.4205 0.9513 -1.7181  2.559  False
     4      5   0.1691 0.9956 -1.8042 2.1424  False
---------------------------------------------------
Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     2      3  -0.3333 0.5618 -0.9952 0.3285  False
     2      4  -0.3529 0.4697 -0.9829  0.277  False
     2      5  -0.7292 0.0353 -1.4233 -0.035   True
     3      4  -0.0196 0.9994 -0.4582  0.419  False
     3      5  -0.3958 0.2124 -0.9225 0.1309  False
     4      5  -0.3762 0.1898 -0.8622 0.1098  False
------------

In [98]:
# czytanie ze zrozumieniem
conduct_chi2test_for_many(dfs, ALPHA, names, 'trud', 'Czyt_ze_zroz')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Czyt_ze_zroz  1  2  3  4  5
trud                       
9             0  0  0  1  0
10            0  1  1  2  0
11            0  2  0  0  1
12            0  0  1  1  0
13            1  2  0  2  0
14            0  0  2  2  2
15            0  1  0  0  2
16            0  0  0  4  1
17            0  0  1  2  0
18            0  0  1  1  0
19            0  0  0  1  1
20            0  1  0  1  1
22            1  0  0  0  0
Wartość statystyki chi-kwadrat: 56.39915966386555
P-value: 0.18971128206013282
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Czyt_ze_zroz  1  2  3  4  5
trud                       
5             1  0  0  1  0
6             0  4  2  5  2
7             0  0  3  1  2
8             0  2  1  7  2
9             0  0  0  0  2
10            0  1  0  1  0
11            0  0  0  2  0
12            1  0  0  0  0
Wartość statystyki chi-kwadrat: 52.523881347

In [99]:
conduct_posthoc_test(df_sumA, 'trud', 'Czyt_ze_zroz')

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     1      2  -1.3571 0.8575 -5.2938 2.5795  False
     1      3  -1.6667  0.754 -5.6755 2.3422  False
     1      4  -0.8529 0.9619 -4.5233 2.8174  False
     1      5     -1.0 0.9453 -4.8816 2.8816  False
     2      3  -0.3095 0.9974 -3.0411 2.4221  False
     2      4   0.5042 0.9641 -1.7008 2.7092  False
     2      5   0.3571 0.9941 -2.1839 2.8982  False
     3      4   0.8137  0.852 -1.5178 3.1452  False
     3      5   0.6667 0.9498  -1.985 3.3183  False
     4      5  -0.1471 0.9996 -2.2521  1.958  False
---------------------------------------------------


In [100]:
# wzrokowiec
conduct_chi2test_for_many(dfs, ALPHA, names, 'trud', 'Wzrokowiec')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Wzrokowiec  2  3  4  5
trud                  
9           0  0  0  1
10          0  1  1  2
11          0  2  0  1
12          0  0  2  0
13          0  2  2  1
14          0  1  2  3
15          0  0  2  1
16          0  1  1  3
17          0  0  2  1
18          1  1  0  0
19          0  0  2  0
20          1  1  0  1
22          0  0  1  0
Wartość statystyki chi-kwadrat: 37.671957671957664
P-value: 0.39264050837491377
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Wzrokowiec  2  3  4  5
trud                  
5           0  0  1  1
6           0  5  2  6
7           0  1  3  2
8           0  2  5  5
9           0  1  1  0
10          1  0  1  0
11          1  0  1  0
12          0  0  1  0
Wartość statystyki chi-kwadrat: 28.565323565323567
P-value: 0.12483435258959752
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane

In [101]:
conduct_posthoc_test(df_a, 'trud', 'Wzrokowiec')
conduct_posthoc_test(df_1, 'trud', 'Wzrokowiec')

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     2      3  -1.2037 0.0444 -2.3865 -0.0209   True
     2      4  -0.8333 0.2307 -1.9723  0.3056  False
     2      5  -1.2381 0.0284 -2.3818 -0.0943   True
     3      4   0.3704 0.4329 -0.2676  1.0083  False
     3      5  -0.0344  0.999 -0.6808   0.612  False
     4      5  -0.4048 0.2438  -0.967  0.1575  False
----------------------------------------------------
Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     2      3  -0.9444 0.2517 -2.2746 0.3857  False
     2      4  -0.7333 0.4402 -2.0142 0.5475  False
     2      5  -0.9643 0.2087 -2.2505 0.3219  False
     3      4   0.2111 0.8664 -0.5063 0.9285  False
     3      5  -0.0198 0.9999 -0.7468 0.7071  False
     4      5   -0.231 0.7727 -0.8632 0.4013  False
--

In [102]:
# słuchowiec
conduct_chi2test_for_many(dfs, ALPHA, names, 'trud', 'Słuchowiec')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Słuchowiec  1  2  3  4  5
trud                     
9           0  0  1  0  0
10          0  1  2  1  0
11          1  0  0  2  0
12          0  1  1  0  0
13          0  1  3  1  0
14          0  1  2  1  2
15          0  0  2  0  1
16          2  2  0  1  0
17          0  1  1  0  1
18          0  0  1  1  0
19          1  1  0  0  0
20          0  0  1  0  2
22          0  0  1  0  0
Wartość statystyki chi-kwadrat: 45.58174603174603
P-value: 0.5724777802474877
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Słuchowiec  1  2  3  4  5
trud                     
5           0  1  0  1  0
6           1  2  6  3  1
7           0  2  2  0  2
8           2  2  5  2  1
9           0  1  0  0  1
10          0  0  1  0  1
11          1  0  0  1  0
12          0  0  1  0  0
Wartość statystyki chi-kwadrat: 23.113553113553117
P-value: 0.7273366225200715
Brak podstaw do 

In [103]:
# kinestetyk
conduct_chi2test_for_many(dfs, ALPHA, names, 'trud', 'Kinestetyk')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Kinestetyk  1  2  3  4  5
trud                     
9           0  0  0  1  0
10          1  2  0  1  0
11          0  1  0  2  0
12          1  0  0  1  0
13          1  1  1  2  0
14          2  4  0  0  0
15          0  3  0  0  0
16          2  3  0  0  0
17          0  2  0  1  0
18          1  0  0  1  0
19          0  0  1  0  1
20          1  0  1  1  0
22          0  1  0  0  0
Wartość statystyki chi-kwadrat: 55.08888888888889
P-value: 0.2242299628611607
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
Kinestetyk  1  2  3  4  5
trud                     
5           0  0  0  2  0
6           1  6  1  5  0
7           1  4  0  1  0
8           6  5  0  1  0
9           1  0  1  0  0
10          0  1  1  0  0
11          0  0  0  1  1
12          0  1  0  0  0
Wartość statystyki chi-kwadrat: 50.91117814647225
P-value: 0.005108429817012458
Istnieją podsta

In [104]:
conduct_posthoc_test(df_sumA, 'trud', 'Kinestetyk')

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     1      2   -0.366 0.9785 -2.2102 1.4781  False
     1      3   0.5556  0.983 -2.4268  3.538  False
     1      4  -1.1778 0.4785 -3.2333 0.8777  False
     1      5   3.2222  0.304 -1.4934 7.9378  False
     2      3   0.9216 0.8769 -1.8799  3.723  False
     2      4  -0.8118 0.6875 -2.5946 0.9711  False
     2      5   3.5882 0.1888 -1.0151 8.1915  False
     3      4  -1.7333 0.4516 -4.6782 1.2115  False
     3      5   2.6667 0.5792  -2.499 7.8323  False
     4      5      4.4 0.0749 -0.2919 9.0919  False
---------------------------------------------------


In [105]:
# MAT - pisał / nie pisał matury rozszerzonej
conduct_chi2test_for_many(dfs, ALPHA, names, 'trud', 'MAT')

-----------------------------------------------
Zsumowane wszystkie grafiki:
MAT   NO_VAL  VAL
trud             
9          0    1
10         2    2
11         1    2
12         1    1
13         2    3
14         3    3
15         1    2
16         3    2
17         3    0
18         0    2
19         0    2
20         1    2
22         1    0
Wartość statystyki chi-kwadrat: 10.101010101010102
P-value: 0.6070994161148735
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
MAT   NO_VAL  VAL
trud             
5          1    1
6          5    8
7          4    2
8          5    7
9          1    1
10         1    1
11         0    2
12         1    0
Wartość statystyki chi-kwadrat: 4.335664335664335
P-value: 0.7404048894011153
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki B:
MAT   NO_VAL  VAL
trud             
3          0    1
4          2    2
5          2    3
6          0    3
7       

In [106]:
# POL - pisał / nie pisał matury rozszerzonej
conduct_chi2test_for_many(dfs, ALPHA, names, 'trud', 'POL')

-----------------------------------------------
Zsumowane wszystkie grafiki:
POL   NO_VAL  VAL
trud             
9          1    0
10         3    1
11         3    0
12         2    0
13         4    1
14         3    3
15         3    0
16         4    1
17         3    0
18         2    0
19         1    1
20         2    1
22         0    1
Wartość statystyki chi-kwadrat: 11.23058542413381
P-value: 0.5092677392911555
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
POL   NO_VAL  VAL
trud             
5          2    0
6         11    2
7          3    3
8         11    1
9          2    0
10         1    1
11         1    1
12         0    1
Wartość statystyki chi-kwadrat: 10.70122231412554
P-value: 0.1521929406538322
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki B:
POL   NO_VAL  VAL
trud             
3          1    0
4          3    1
5          5    0
6          3    0
7        

In [107]:
# HIST - pisał / nie pisał matury rozszerzonej
conduct_chi2test_for_many(dfs, ALPHA, names, 'trud', 'HIST')

-----------------------------------------------
Zsumowane wszystkie grafiki:
HIST  NO_VAL  VAL
trud             
9          1    0
10         4    0
11         3    0
12         2    0
13         5    0
14         4    2
15         3    0
16         4    1
17         3    0
18         2    0
19         2    0
20         2    1
22         1    0
Wartość statystyki chi-kwadrat: 8.888888888888888
P-value: 0.7123903776087557
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki A:
HIST  NO_VAL  VAL
trud             
5          2    0
6         13    0
7          5    1
8         10    2
9          2    0
10         1    1
11         2    0
12         1    0
Wartość statystyki chi-kwadrat: 6.666666666666668
P-value: 0.46439128108585215
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Zsumowane grafiki B:
HIST  NO_VAL  VAL
trud             
3          1    0
4          4    0
5          5    0
6          2    1
7       

In [108]:
# dominujący typ uczenia poznawczego
conduct_chi2test_for_many(dfs, ALPHA, names, 'trud','Typ_uczenia')

-----------------------------------------------
Zsumowane wszystkie grafiki:
Typ_uczenia  Brak dominującego  Kinestetyk  Słuchowiec  Wzrokowiec
trud                                                              
9                            0           0           0           1
10                           0           1           0           3
11                           1           0           1           1
12                           1           0           0           1
13                           3           0           0           2
14                           2           0           1           3
15                           0           0           1           2
16                           1           0           0           4
17                           0           0           1           2
18                           2           0           0           0
19                           0           1           0           1
20                           0           0          

Podsumowanie - trudność:
Istotne statystycznie:
- wady wzroku: zsumowane grafiki 2 (mało różnicuje)
- ilość snu: zsumowane grafiki 3 (mało różnicuje), wszystkie grafiki (mało różnicuje), grafiki 3 (widoczna różnica <6 a >8)
- matura matematyka podstawowa: zsumowane grafiki B, zsumowane grafiki 1 (mało różnicuje)
- sampoczoucie: zsumowane grafiki B, grafiki 2, 3 (podobne wnioski jak poprawność)
- zmęczenie: wszystkie grafiki (1 vs 4, 2 vs 4, 3 vs 4)
- podzielność uwagi: zsumowane grafiki 3 (słabo się różnicuje), grafiki 3 (1 vs 4, 1 vs 3, 1 vs 2)
- spostrzegawczość: zsumowane grafiki 2, grafiki wszystkie, A, 3 (głównie 2 vs 5) --> fajny wniosek dla grafik 2, widać sporzegawczośc vs. rozkminienie że zadanie było proste 
- czytanie ze zrozumieniem: zsumowane grafiki A (słabo się różnicuje)
- wzrokowiec: grafiki A (2 vs 3, 2 vs 5), grafiki 1 (słabo się różnicuje)
- kinestetyk: zsumowane grafiki A (słabo się różnicuje)